In [9]:
import PyPDF2
import csv
import json
import pandas as pd
import matplotlib.pyplot as plt
import camelot
import tabula
import requests
import os
import re

In [82]:
def download(url, file_name):
    # open in binary mode
    with open(file_name, "wb") as file:
        # get request
        response = requests.get(url)
        # write to file
        file.write(response.content)
#iterate over df and access links, download pdfs, convert tables to csv
def get_tables(df,start=0, end=-1):
    for row in df[start:end].values:
        for i in row:
            if '.pdf' in str(i):
                if '200' in str(requests.get(str(i))):
                    download(str(i), str(row[1])+'.pdf')
                    pages = lookup_table(str(row[1])+'.pdf')
                    tables = tabula.read_pdf(str(row[1])+'.pdf',multiple_tables=True, pages=pages)
                    x=1
                    for t in tables:
                        
                        t['jusrisdiction'] = str(row[1])
                        t['location_type'] = str(row[2])
                        t['table_number'] = x
#                         t.rename_axis('table number '+str(x), axis=1, inplace=True)
                        x += 1
                    return tables
#                     tables = tabula.convert_into(str(row[1])+'.pdf', str(row[1])+'.csv',output_format='csv',
#                                              multiple_tables=True, pages='all')
#                     os.remove(str(row[1])+'.pdf')


In [81]:
def lookup_table(pdf_doc):
    string = r'[T|t]{1}able\s\d+'
#     string = r'\w+'
    doc = PyPDF2.PdfFileReader(open(pdf_doc, 'rb'))
    pages = []
    for i in range(doc.getNumPages()):
        page = doc.getPage(i)
        text = page.extractText()
        if re.search(string, text):
#             print(i)
            pages.append(i)  
    return pages

In [84]:
#Load list of urls to extract pdfs from
df = pd.read_csv('Cities.csv')
get_tables(df,17,19)[2]

,0,1,2,3,4,5,6,jusrisdiction,location_type,table_number
0,Strategy,Lifecycle Costs*,NaN,Lifecycle Cost Unit,Investment Lifetime,NaN,Payback Period,Artesia,City,2
1,Low Water Sanitary Fixtures,"-$865,382",NaN,For all bus and rail facilities,20 years,NaN,3 years,Artesia,City,2
2,Retrofit Lighting in Red Line Tunnel,"-$487,692",NaN,For all tunnel fixtures,4 years,NaN,2 years,Artesia,City,2
3,Building Indoor Lighting Upgrades: LEDs,"-$321,265",NaN,For all Divisions,6 years,NaN,2 years,Artesia,City,2
4,Extension of Bus Wash On-Site Reclamation,"-$274,237",NaN,Per bus washing facility,40 years,NaN,1 year,Artesia,City,2
5,Building Indoor Lighting Upgrades: Efficient M...,"-$133,750",NaN,For all Divisions,3 years,NaN,1 year,Artesia,City,2
6,Municipal Recycled Water For Bus Washing,"-$123,894",NaN,Per bus washing facility,40 years,NaN,1 year,Artesia,City,2
